# 🐾 Animal Detection Model - GUI Based (Tkinter + YOLOv8)

In [ ]:

# 🐾 Animal Detection Model with GUI (Tkinter + YOLOv8)
# Detects animals in images or videos, highlights carnivores in red, and shows a count popup

from ultralytics import YOLO
import cv2
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import os

# Load YOLOv8 model (place your trained model in models/best.pt)
model = YOLO("models/best.pt")

# Define carnivores
CARNIVORES = {"lion", "tiger", "leopard", "wolf", "cheetah", "hyena", "fox"}

# Detection Function
def detect_animals(path, is_video=False):
    carnivore_count = 0

    if is_video:
        cap = cv2.VideoCapture(path)
        width, height = int(cap.get(3)), int(cap.get(4))
        fps = cap.get(cv2.CAP_PROP_FPS)
        out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            detections = model(frame)[0]
            for box in detections.boxes.data:
                x1, y1, x2, y2, conf, cls = box
                label = detections.names[int(cls)]
                color = (0, 255, 0)
                if label in CARNIVORES:
                    color = (0, 0, 255)
                    carnivore_count += 1
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
                cv2.putText(frame, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

            out.write(frame)

        cap.release()
        out.release()
        return "output.mp4", carnivore_count

    else:
        img = cv2.imread(path)
        detections = model(img)[0]
        for box in detections.boxes.data:
            x1, y1, x2, y2, conf, cls = box
            label = detections.names[int(cls)]
            color = (0, 255, 0)
            if label in CARNIVORES:
                color = (0, 0, 255)
                carnivore_count += 1
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)
            cv2.putText(img, label, (int(x1), int(y1)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)

        output_path = "output.jpg"
        cv2.imwrite(output_path, img)
        return output_path, carnivore_count

# GUI Class
class AnimalDetectionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Animal Detection Model")
        self.img_label = tk.Label(root)
        self.img_label.pack()

        tk.Button(root, text="Detect in Image", command=self.open_image).pack(pady=10)
        tk.Button(root, text="Detect in Video", command=self.open_video).pack(pady=10)

    def open_image(self):
        path = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.png *.jpeg")])
        if path:
            output_path, count = detect_animals(path, is_video=False)
            self.display_image(output_path)
            messagebox.showinfo("Carnivores Detected", f"Number of carnivorous animals: {count}")

    def open_video(self):
        path = filedialog.askopenfilename(filetypes=[("Videos", "*.mp4 *.avi")])
        if path:
            output_path, count = detect_animals(path, is_video=True)
            messagebox.showinfo("Carnivores Detected", f"Number of carnivorous animals: {count}")
            os.system(f"start {output_path}" if os.name == 'nt' else f"xdg-open {output_path}")

    def display_image(self, path):
        img = Image.open(path).resize((640, 480))
        img = ImageTk.PhotoImage(img)
        self.img_label.config(image=img)
        self.img_label.image = img

# To run the GUI (in local Jupyter Notebook or script):
# root = tk.Tk()
# app = AnimalDetectionApp(root)
# root.mainloop()
